In [132]:
import requests
from bs4 import BeautifulSoup
import json
import pandas

def hente_tabell(_):
    # URL til nettsiden du vil trekke ut data fra
    url = _["url"]
    # Send en GET-forespørsel for å hente nettsiden
    response = requests.get(url)

    # Sjekk om forespørselen var vellykket
    if response.status_code == 200:
        # Analyser HTML-innholdet med Beautiful Soup
        soup = BeautifulSoup(response.content, "html.parser")

        # Finn tabellen som inneholder dataen
        table = soup.find("table", attrs={"id":"table-id"})

        # Hent ut tabellens overskrifter
        headers = [header.text.strip() for header in table.find_all("th")]

        # Hent ut dataene fra tabellen
        data = []
        for row in table.find_all("tr")[1:]:
            row_data = [td.text.strip() for td in row.find_all("td")]
            row_data.append(_["type"])
            headers.append("type")
            data.append(dict(zip(headers, row_data)))
            
        return data
    else:
        print("Feil: Kan ikke hente nettsiden")
        return None

In [133]:
urls = [{"type":"Kollektiv beskyttelse", "url":"https://www.udi.no/statistikk-og-analyse/statistikk/soknader-om-beskyttelse-etter-hjemmel-om-massefluktsituasjon--kollektiv-beskyttelse-etter-statsborgerskap-og-maned-2024/"},
 {"type":"Kollektiv Beskyttelse EMA", "url":"https://www.udi.no/statistikk-og-analyse/statistikk/soknader-om-beskyttelse-etter-hjemmel-om-massefluktsituasjon-kollektiv--beskyttelse-fra-enslige-mindrearige-etter-soknadsmaned-og-statsborgerskap-2024/"},
 {"type":"Asylsoknader EMA", "url":"https://www.udi.no/statistikk-og-analyse/statistikk/asylsoknader-fra-enslige-mindrearige-etter-soknadsmaned-og-statsborgerskap-2024/"},
 {"type":"Asylsoknader","url":"https://www.udi.no/statistikk-og-analyse/statistikk/asylsoknader-etter-statsborgerskap-og-maned-2024/"}]
 

In [134]:
tabs = []
for x in urls:
    tab = hente_tabell(x)
    for row in tab:
        tabs.append(row)

In [136]:
d = pandas.DataFrame(tabs)

In [137]:
d["Total"] = d.Total.apply(int)

In [138]:
d.groupby("type").Total.sum()

type
Asylsoknader                  2656
Asylsoknader EMA               178
Kollektiv Beskyttelse EMA      432
Kollektiv beskyttelse        11372
Name: Total, dtype: int64